In [40]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from typing import TypedDict
from dotenv import load_dotenv

In [41]:
load_dotenv()

llm = ChatOpenAI()

In [42]:
# create a state 

class BlogState(TypedDict):

    title: str 
    outline: str
    content: str 
    evaluate: str
    rating: float 




In [43]:
# --- 3️⃣ Define Node Functions ---
def create_outline(state: BlogState):
    """Generate an outline based on the blog title."""
    title = state["title"]
    prompt = f"Create a detailed blog outline for the title: '{title}'"
    response = llm.invoke(prompt)
    return {"outline": response.content.strip()}

In [44]:
def create_blog(state: BlogState):
    """Generate blog content from the outline."""
    outline = state["outline"]
    prompt = f"Write a well-structured blog post following this outline:\n{outline}"
    response = llm.invoke(prompt)
    return {"content": response.content.strip()}

In [45]:
def evaluate_blog(state: BlogState):
    """Evaluate blog content based on how well it follows the outline."""
    outline = state["outline"]
    content = state["content"]
    prompt = f"""
Evaluate the following blog based on its alignment with the outline.
Rate from 1 to 10 and give a short justification.

Outline:
{outline}

Content:
{content}
"""
    response = llm.invoke(prompt)
    return {"rating": response.content.strip()}


In [46]:
# build a graph 
graph = StateGraph(BlogState)

# Nodes
graph.add_node("create_outline", create_outline)
graph.add_node("create_blog", create_blog)
graph.add_node("evaluate_blog", evaluate_blog)

# Edges (Prompt chaining)
graph.add_edge(START, "create_outline")
graph.add_edge("create_outline", "create_blog")
graph.add_edge("create_blog", "evaluate_blog")
graph.add_edge("evaluate_blog", END)



workflow = graph.compile()

In [47]:
initial_state = {"title": "How AI Agents Are Changing Software Development"}
final_state = workflow.invoke(initial_state)


In [48]:
print("\n--- OUTLINE ---")
print(final_state["outline"])
print("\n--- CONTENT ---")
print(final_state["content"])
print("\n--- EVALUATION ---")
print(final_state["rating"])


--- OUTLINE ---
I. Introduction
 - Brief overview of artificial intelligence (AI) and its impact on various industries
 - Introduction to AI agents in software development
 
II. What are AI Agents?
 - Definition of AI agents in the context of software development
 - Explanation of how AI agents differ from traditional software development methods
 
III. Benefits of AI Agents in Software Development
 - Increased efficiency and productivity
 - Faster development times
 - Improved accuracy and quality of code
 
IV. Examples of AI Agents in Action
 - Case studies of successful AI agents in software development
 - Overview of popular tools and platforms using AI agents
 
V. Challenges and Limitations of AI Agents in Software Development
 - Potential risks and drawbacks of relying on AI agents
 - Ethical considerations and concerns
 
VI. Future of AI Agents in Software Development
 - Predictions for how AI agents will continue to shape the software development landscape
 - Potential advance